<a href="https://colab.research.google.com/github/CloudsDocker/AI_manage_stripe_payment/blob/main/seq2seq_lei_zhang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run below command to install libraries

In [ ]:
!pip3 install torch numpy matplotlib

Mound and load data files ( to mount the files needed)

In [7]:
import os
from google.colab import drive

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Path to the zip file containing training data
data_file_path = "drive/MyDrive/Cooking_Dataset"
data_file_path = '/content/drive/MyDrive/Cooking_Dataset'  # Google drive

train_data_path = os.path.join(data_file_path, "train")

print(os.path.exists(train_data_path))
MAX_FILES = 10

Using device: cuda
True


In [ ]:
import os
import random
import re
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Define the self-attention mechanism
# class SelfAttention(nn.Module):
#     def __init__(self, hidden_size):
#         super(SelfAttention, self).__init__()
#         self.hidden_size = hidden_size

#         # Define layers for Q, K, and V projections
#         self.linear_q = nn.Linear(hidden_size, hidden_size)
#         self.linear_k = nn.Linear(hidden_size, hidden_size)
#         self.linear_v = nn.Linear(hidden_size, hidden_size)

#     def forward(self, query, key, value):
#         # Project input tensors
#         Q = self.linear_q(query)
#         K = self.linear_k(key)
#         V = self.linear_v(value)

#         # Compute attention scores
#         scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.hidden_size).float())

#         # Apply softmax to obtain attention weights
#         attention_weights = F.softmax(scores, dim=-1)

#         # Compute attention output
#         attention_output = torch.matmul(attention_weights, V)

#         return attention_output

# Define the Seq2Seq model with self-attention
# class Seq2Seq_WithAttn(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(Seq2Seq, self).__init__()
#         self.hidden_size = hidden_size
#
#         # Define encoder layers
#         self.encoder_embedding = nn.Embedding(input_size, hidden_size)
#         self.encoder_self_attention = SelfAttention(hidden_size)
#
#         # Define decoder layers
#         self.decoder_embedding = nn.Embedding(output_size, hidden_size)
#         self.decoder_self_attention = SelfAttention(hidden_size)
#
#         # Define fully connected layer to output vocabulary
#         self.fc = nn.Linear(hidden_size, output_size)
#
#     def forward(self, input_seq, target_seq):
#         # Encoder forward pass
#         encoder_embedded = self.encoder_embedding(input_seq)
#         encoder_output = self.encoder_self_attention(encoder_embedded, encoder_embedded, encoder_embedded)
#
#         # Decoder forward pass
#         decoder_embedded = self.decoder_embedding(target_seq)
#         decoder_output = self.decoder_self_attention(decoder_embedded, decoder_embedded, decoder_embedded)
#
#         # Apply fully connected layer to decoder output
#         output = self.fc(decoder_output)
#
#         return output

from typing import List, Tuple
def preprocess_data(ingredients: List[str], instructions: List[str]) -> Tuple[List[torch.Tensor], List[torch.Tensor]]:
    """
    Preprocesses the recipe data by tokenizing and converting to tensors.
    Returns lists of ingredient and instruction tensors.
    """
    # tokenizer = get_tokenizer('basic_english')

    # Extract just the ingredient names, ignoring the numbers and units
    ingredient_names = []
    for ingr_text in ingredients:
        ingr_parts = ingr_text.split(',')
        ingr_names = [re.sub(r'\d+\s*\w*', '', part.strip()) for part in ingr_parts]
        ingredient_names.append(' '.join(ingr_names))



def load_data_from_all_files(data_dir, max_files=None):
    ingredients = []
    instructions = []

    for file_name in os.listdir(data_dir):
        if max_files and len(ingredients) >= max_files:
            break
        with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as file:
            print(f"Reading file: {file_name}")
            recipe_parts = file.read().split('END RECIPE\n')
            for recipe in recipe_parts:
                lines = recipe.split('\n')
                ingr_line = next((line for line in lines if line.startswith('ingredients:')), None)
                if ingr_line:
                    ingr_text = ingr_line.split(':')[1].strip()
                    # ingr_items = ingr_line.split(':')[1].strip().split("\t")
                    # ingr_text = ' '.join(ingr_items)

                    # to get the instructions from following lines until the end of the recipe
                    instr_start = lines.index(ingr_line)
                    instr_text = '. '.join(line.strip() for line in lines[instr_start+1:])

                    if ingr_text and instr_text:
                        ingredients.append(ingr_text)
                        instructions.append(instr_text)

    return ingredients, instructions

# Function to load data from zip file
# def load_data_from_zip(zip_file_path):
#     data = []
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         for file_info in zip_ref.infolist():
#             with zip_ref.open(file_info) as file:
#                 # Assuming each file contains input-output pairs separated by newline
#                 pairs = file.read().decode('utf-8').strip().split('\n')
#                 data.extend([pair.split('\t') for pair in pairs])
#     return data
# Function to convert text to tensor
def text_to_tensor(text, vocab):
    tensor = [vocab[token] for token in text.split()]
    return torch.tensor(tensor, dtype=torch.long).to(device)

# Data preprocessing
# tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter, tokenizer):
    for text in data_iter:
        if isinstance(text, list):
            yield tokenizer(' '.join(text))
        else:
            yield tokenizer(text)



# Load training data from the zip file
ingredients, instructions = load_data_from_all_files(train_data_path, MAX_FILES) # total list of ingredients and instructions


def preprocess_data(ingredients: List[str], instructions: List[str]) -> Tuple[List[torch.Tensor], List[torch.Tensor]]:
    """
    Preprocesses the recipe data by tokenizing and converting to tensors.
    Returns lists of ingredient and instruction tensors.
    """
    tokenizer = get_tokenizer('basic_english')

    # Normalize ingredient text
    normalized_ingredients = []
    for ingr_text in ingredients:
        ingr_parts = re.split(r'[;,]', ingr_text)
        ingr_names = [re.sub(r'\d+\s*\w*', '', part.strip()) for part in ingr_parts]
        normalized_ingredient = ' '.join(ingr_names)
        normalized_ingredients.append(normalized_ingredient)

    vocab_ingr = build_vocab_from_iterator(yield_tokens(normalized_ingredients, tokenizer), specials=['<unk>', '<pad>'])
    vocab_instr = build_vocab_from_iterator(yield_tokens(instructions, tokenizer), specials=['<unk>', '<pad>'])

    ingr_tensors = [torch.tensor(vocab_ingr.lookup_indices(tokenizer(ingr)), dtype=torch.long, device=device) for ingr in normalized_ingredients]
    instr_tensors = [torch.tensor(vocab_instr.lookup_indices(tokenizer(instr)), dtype=torch.long, device=device) for instr in instructions]

    return ingr_tensors, instr_tensors, vocab_ingr, vocab_instr # the tensor representation of ingredients and instructions, same length as the input


#TODO: to split by tab and space then build vocab
# to load data
# vocab_ingr = build_vocab_from_iterator(yield_tokens(ingredients), specials=['<unk>'])
# vocab_instr = build_vocab_from_iterator(yield_tokens(instructions), specials=['<unk>'])
#
# # def build_vocab_by_tokenizer(data, tokenizer, specials=None):
#
#
# train_data = [vocab_ingr(ingr) for ingr in ingredients]
# train_data = [vocab_instr(instr) for instr in instructions]
#
# # Create datasets and data loaders
# train_dataset = to_map_style_dataset(list(zip(train_data, train_data)))
#
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

ingr_tensors, instr_tensors, vocab_ingr, vocab_inst = preprocess_data(ingredients, instructions)
# Now you can use ingr_tensors and instr_tensors to train your neural network
# For example, you can create a PyTorch dataset and dataloader:
# train_dataset = torch.utils.data.TensorDataset(ingr_tensors, instr_tensors)
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)


import torch.nn.utils.rnn as rnn_utils
# Model architecture
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers, dropout):
        super().__init__()
        self.hid_dim = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size,  dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        # print("src_lengths:", src_lengths)
        # print("Input shape: %s and type: %s and data: %s" % (src.shape, src.type(), src))
        # Add a batch dimension to the input tensor if it's 1-dimensional
        # if len(src.shape) == 1:
        #     print("Shape is 1-dimensional")
        #     src = src.unsqueeze(0)  # Assuming src is a 1-dimensional tensor
        # else:
        #     print("Shape is not 1-dimensional, the dimension is:", len(src.shape))
        # print("After unsqueeze shape:", src.shape)

        # if src.is_cuda:
        #   print("src is on GPU")
        # else:
        #   print("src is on CPU")
        # print(f"embedding layer device: {self.embedding.weight.device}")
        # print(f"dropout layer device: {self.dropout.device}")  # Assuming 'dropout' is a layer


        embedded = self.dropout(self.embedding(src))
        # embedded = embedded.unsqueeze(0)
        # print("Embedded shape:", embedded.shape)
        # packed_embedded = rnn_utils.pack_padded_sequence(embedded, src_lengths, enforce_sorted=False)

        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, input_size, emb_dim, hidden_size, n_layers, dropout):
        super().__init__()
        self.output_dim = input_size
        self.hid_dim = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_size, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_size,1,  dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, input_size)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        # Add batch dimension to input
        # input = input.unsqueeze(0)

        # Add batch dimension to hidden and cell states
        # hidden = hidden.unsqueeze(0)
        # cell = cell.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))
        # embedded = embedded.unsqueeze(0)
        # print("shape of embedded:", embedded.shape)
        # print("shape of hidden:", hidden.shape)
        # print("shape of cell:", cell.shape)
        # Add batch dimension to hidden and cell states
        # Add batch dimension and sequence dimension to hidden and cell states
        # hidden = hidden.unsqueeze(0).unsqueeze(0)  # (1, 1, 256)
        # cell = cell.unsqueeze(0).unsqueeze(0)  # (1, 1, 256)
        # print("shape of embedded:", embedded.shape)
        # print("shape of hidden:", hidden.shape)
        # print("shape of cell:", cell.shape)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # print("shape of output:", output.shape)
        # print(("value of output:", output))
        # Remove batch dimension from output
        prediction = self.fc_out(output.squeeze(0))
        # prediction = self.softmax(self.fc_out(output[0]))
        # print("shape of prediction:", prediction.shape)
        return prediction, hidden, cell


# class Seq2Seq(nn.Module):
#     def __init__(self, encoder, decoder, device):
#         super().__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#         self.device = device
#
#     def forward(self, src, trg, teacher_forcing_ratio=TEACHER_FORCING_RATIO):
#         batch_size = src.shape[1]
#         trg_len = trg.shape[0]
#         trg_vocab_size = self.decoder.output_dim
#
#         outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
#         hidden, cell = self.encoder(src)
#
#         input = trg[0, :]
#         for t in range(1, trg_len):
#             output, hidden, cell = self.decoder(input, hidden, cell)
#             outputs[t] = output
#             teacher_force = random.random() < teacher_forcing_ratio
#             top1 = output.argmax(1)
#             input = trg[t] if teacher_force else top1
#
#         return outputs
def checkDevice(data, text):
  if data.is_cuda:
    print(f"{text} is on GPU")
  else:
    print(f"{text} is on CPU")

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()

    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

# Hyperparameters
INPUT_DIM = len(ingr_tensors)
OUTPUT_DIM = len(instr_tensors)
print("ingr_tensors length:", len(ingr_tensors))
print("instr_tensors length:", len(instr_tensors))
# checkDevice(ingr_tensors[0],"ingr_tensors")
# checkDevice(ingr_tensors[0],"instr_tensors")
# ingr_tensors length: 3289
# instr_tensors length: 3289
ENC_EMB_DIM = len(vocab_ingr)
DEC_EMB_DIM = len(vocab_inst)
print("ENC_EMB_DIM:", ENC_EMB_DIM)
print("DEC_EMB_DIM:", DEC_EMB_DIM)

hidden_size = 256
N_LAYERS = 1
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
MAX_LENGTH = 100
LEARNING_RATE = 0.01
TEACHER_FORCING_RATIO = 1.0


# ========  declare variables ========

SOS_token = 0
EOS_token = 1

# Instantiate models
enc = Encoder( ENC_EMB_DIM, 128, hidden_size, N_LAYERS, ENC_DROPOUT).to(device)
dec = Decoder( DEC_EMB_DIM, 128,hidden_size, N_LAYERS, DEC_DROPOUT).to(device)

# model = Seq2Seq(enc, dec, device).to(device)

# Define optimization and loss function
# optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()

# Training loop
# N_EPOCHS = 10

encoder_optimizer = optim.Adam(enc.parameters(), lr=LEARNING_RATE)
decoder_optimizer = optim.Adam(dec.parameters(), lr=LEARNING_RATE)
loss_total_print = 0
loss_total_plot = 0
plot_losses = []
print_every = 10
plot_every = 30

for epoch in range(MAX_LENGTH):
    train_loss = 0
    start_time = time.time()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # to get a random index of INPUT_DIM
    random_data_idx = random.randint(0, INPUT_DIM)
    input_tensor = ingr_tensors[random_data_idx]
    target_tensor = instr_tensors[random_data_idx]

    # Calculate the lengths of the sequences in input_tensor
    # src_lengths = [len(seq) for seq in input_tensor]

    # Create a tensor of sequence lengths
    # src_lengths_tensor = torch.tensor(src_lengths).to(device)

    # ==== encoder loop ====
    # loop of each input tensor in encoder
    # Iterate over each sequence in input_tensor
    for src in input_tensor:
        # Ensure src has at least one dimension
        if src.dim() == 0:
            src = src.unsqueeze(0)

        # Get the length of the sequence
        seq_length = src.size(0)

        # print(f"src device: {src.device}")
        # print(f"embedding layer device: {Encoder.embedding.weight.device}")  # Assuming 'embedding' is a layer
        # print(f"dropout layer device: {Encoder.dropout.weight.device}")  # Assuming 'dropout' is a layer

        # Pass src and its length to the encoder
        hidden, cell = enc(src.unsqueeze(0), torch.tensor([seq_length]).to(device))



        # hidden = hidden.squeeze(0)
        # cell = cell.squeeze(0)

    decoder_input = torch.tensor([[SOS_token]], device=device)  # <SOS> token
    # decoder_hidden = hidden

    # ====== decoder loop ======
    # print("target_tensor shape:", target_tensor.shape)
    # print("target value at index idx:", target_tensor[random_data_idx])
    target_length = len(target_tensor)
    # loop of each target tensor in decoder
    # target_tensor = target_tensor.unsqueeze(0).to(device)  # Reshape target for batch size 1

    # Reshape target for teacher forcing
    target_tensor = target_tensor.unsqueeze(0)  # Batch dimension
    target_tensor = torch.nn.functional.pad(target_tensor, (1, 0), value=EOS_token)  # Pad with EOS token

    for i in range(target_length):
        # trg = target_tensor[i]
        # trg=trg.unsqueeze(0)
        # print("i:", i)
        trg = target_tensor[:, i]  # Access current element within the batch

        # trg = target_tensor[:, i]  # Access current element within the batch

        output, hidden, cell = dec(decoder_input, hidden, cell)
        # print("output shape:", output.shape)
        # print("target_tensor shape:", target_tensor.shape)
        # print("trg shape:", trg.shape)
        # print("trg value:", trg)
        train_loss += criterion(output, trg)
        # print("train_loss:", train_loss)
        # check if we need to use teacher forcing
        teacher_force = random.random() < TEACHER_FORCING_RATIO
        if teacher_force:
            decoder_input = trg.unsqueeze(0)
        else:
            topv, topi = output.topk(1)
            decoder_input = topi.squeeze().detach()
            if decoder_input.item() == EOS_token:
                break

    train_loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    #
    # # for batch in train_dataloader:
    # #     src = batch[0].to(device)
    # #     trg = batch[1].to(device)
    #
    # optimizer.zero_grad()
    #
    # output = model(src, trg)
    #
    # output_dim = output.shape[-1]
    #
    # output = output[1:].view(-1, output_dim)
    # trg = trg[1:].reshape(-1)
    #
    # loss = criterion(output, trg)
    #
    # loss.backward()
    #
    # optimizer.step()
    #
    # train_loss += loss.item()
    #
    # train_loss /= len(train_dataloader)
    loss_avg = train_loss.item() / target_length
    loss_total_print += loss_avg
    loss_total_plot += loss_avg
    if epoch % print_every == 0:
        print(f"Epoch: {epoch} | Loss: {loss_avg:.3f}")
        loss_total_print = 0
    if epoch % plot_every == 0:
        plot_loss_avg = loss_total_plot / plot_every
        plot_losses.append(plot_loss_avg)
        loss_total_plot = 0


    end_time = time.time()
    epoch_mins = int((end_time - start_time) // 60)
    epoch_secs = int((end_time - start_time) % 60)


    print(f'Epoch: {epoch + 1}/{MAX_LENGTH} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')

showPlot(plot_losses)
#
# # Define input and output vocabulary sizes
# input_size = 10  # Example: Size of input vocabulary
# output_size = 10  # Example: Size of output vocabulary
#
# # Define hidden size
# hidden_size = 256  # Example: Size of hidden layers
#
# # Initialize Seq2Seq model
# model = Seq2Seq(input_size, hidden_size, output_size)
#
# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

#
#
# # Example training loop (assuming input and target sequences are tensors)
# num_epochs = 10  # Example: Number of epochs
# for epoch in range(num_epochs):
#     for input_text, target_text in training_data:
#         # Convert input and target texts to tensors (you need to implement this)
#         input_seq = text_to_tensor(input_text)
#         target_seq = text_to_tensor(target_text)
#
#         optimizer.zero_grad()
#
#         # Forward pass
#         output = model(input_seq, target_seq)
#
#         # Calculate loss
#         loss = criterion(output.view(-1, output_size), target_seq.view(-1))
#
#         # Backpropagation
#         loss.backward()
#         optimizer.step()
#
#     # Example: Print loss after each epoch
#     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
